# Country Classification

### 国際機関のCountry Classificationについて

International Monetary Fund (IMF)<br>
> World Economic Outlook<br>https://www.imf.org/external/pubs/ft/weo/2020/01/weodata/groups.htm
> * The country classification in the World Economic Outlook divides the world into two major groups: `advanced economies` and `emerging and developing economies`. Please refer to the introduction to the WEO's Statistical Appendix for details. This classification is not based on strict criteria, economic or otherwise, but instead has evolved over time with the objective of facilitating analysis by providing a reasonably meaningful organization of the data.<br>
> * Some countries remain outside the country classification and therefore are not included in the analysis. `Anguilla`, `Cuba`, `the Democratic People’s Republic of Korea`, and `Montserrat` are examples of countries that are not IMF members, and their economies therefore are not monitored by the IMF.<br>

World Bank<br>
> World Bank Country and Lending Groups<br>https://datahelpdesk.worldbank.org/knowledgebase/articles/906519-world-bank-country-and-lending-groups
> * Economies are currently divided into four income groupings: `low`, `lower-middle`, `upper-middle`, and `high`. Income is measured using gross national income (GNI) per capita, in U.S. dollars, converted from local currency using the World Bank Atlas method. Estimates of GNI are obtained from economists in World Bank country units; and the size of the population is estimated by World Bank demographers from a variety of sources, including the UN’s biennial World Population Prospects.<br>
> * Countries are immediately reassigned on July 1 each year, based on the estimate of their GNI per capita for the previous calendar year. Income groupings remain fixed for the entire fiscal year (i.e., until July 1 of the following year), even if GNI per capita estimates are revised in the meantime.

United Nations<br>
> World Economic Situation and Prospects (WESP)<br>https://www.un.org/development/desa/publications/wesp-2020.html
> * For analytical purposes, the WESP classifies all countries of the world into one of three broad categories: `developed economies`, `economies in transition` and `developing economies`. The composition of these analytical groupings is intended to reflect basic economic country conditions, and are not strictly aligned with the regional classifications defined by the Statistics Division of UN DESA known as M49.<br>
> * Several countries (in particular the economies in transition) have characteristics that could place them in more than one category; however, for purposes of analysis, the groupings have been made mutually exclusive. Within each broad category, some subgroups are defined based either on geographical location or on ad hoc criteria, such as the subgroup of “major developed economies”, which is based on the membership of the Group of Seven.<br>


> Standard country or area codes for statistical use (M49)<br>https://unstats.un.org/unsd/methodology/m49/<br>
> * Although there is no established convention for the designation of `"developed"` and `"developing"` countries or areas in the United Nations system, in 1996 this concept was introduced to the Standard country or area codes for statistical use (known as M49). It is intended for statistical convenience and does not express a judgement about the stage reached by a particular country or area in the development process. The distinction remains relevant to the Sustainable Development Goals which currently uses for global reporting the definition used in the final report of the Millennium Development Goals (MDG). From 2018, the M49 adopts the same definition.

Others<br>
> ISO 3166 Country Code<br>
https://www.iso.org/obp/ui/

運用上の注意<br>
* M49の`developed`のうち、WESPの`economies in transition`は`developing`に含める<br>
Belarus, Republic of Moldova, Russian Federation, Ukraine, Albania, Bosnia and Herzegovina, Montenegro, North Macedonia, Serbia
* Israelは`developed`に含める（WESPでは`developing`）

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

## IMF

In [2]:
response = requests.get('https://www.imf.org/external/pubs/ft/weo/2020/01/weodata/groups.htm')
doc = BeautifulSoup(response.content,'html.parser')

In [3]:
doc.find_all('table')[4].find(class_='center').text

'Advanced Economies'

In [4]:
advanced = []
for countries in doc.find_all('table')[4].find_all('p'):
    advanced.extend(countries.text.split('\r\n'))

In [5]:
advanced = list(filter(None,advanced))

In [6]:
doc.find_all('table')[9].find(class_='center').text

'Emerging and Developing Economies'

In [7]:
emerging = []
for countries in doc.find_all('table')[9].find_all('p'):
    emerging.extend(countries.text.split('\r\n'))

In [8]:
emerging = list(filter(None,emerging))

In [9]:
IMF = pd.concat([pd.DataFrame({'Country':advanced,'Classification':'Advanced Economies'}),
                 pd.DataFrame({'Country':emerging,'Classification':'Emerging and Developing Economies'})],ignore_index=True)

In [10]:
IMF.head()

,Country,Classification
0,Australia,Advanced Economies
1,Austria,Advanced Economies
2,Belgium,Advanced Economies
3,Canada,Advanced Economies
4,Cyprus,Advanced Economies


In [11]:
len(IMF)

194

In [12]:
IMF.Country = IMF.Country.str.replace('\d','')

In [13]:
IMF.Country = IMF.Country.str.replace('\xa0',' ')

In [14]:
codes = pd.read_csv('IMF_Codes.csv')

In [15]:
len(codes)

194

In [16]:
codes.head()

,IMF Code,ISO Code,Country
0,512,AFG,Afghanistan
1,914,ALB,Albania
2,612,DZA,Algeria
3,614,AGO,Angola
4,311,ATG,Antigua and Barbuda


In [17]:
IMF['ISO Code'] = IMF.Country.map(codes.set_index('Country')['ISO Code'].to_dict())

In [18]:
IMF.head()

,Country,Classification,ISO Code
0,Australia,Advanced Economies,AUS
1,Austria,Advanced Economies,AUT
2,Belgium,Advanced Economies,BEL
3,Canada,Advanced Economies,CAN
4,Cyprus,Advanced Economies,CYP


In [19]:
IMF.to_csv('IMF_Country_Classification.csv')

## World Bank

In [20]:
WB = pd.read_excel('http://databank.worldbank.org/data/download/site-content/CLASS.xls',skiprows=4,skipfooter=56,usecols=[2,3,5,6]
                  ).drop(0).reset_index(drop=True)

In [21]:
WB.head()

,Economy,Code,Region,Income group
0,Afghanistan,AFG,South Asia,Low income
1,Albania,ALB,Europe & Central Asia,Upper middle income
2,Algeria,DZA,Middle East & North Africa,Upper middle income
3,American Samoa,ASM,East Asia & Pacific,Upper middle income
4,Andorra,AND,Europe & Central Asia,High income


In [22]:
len(WB)

218

In [23]:
IMF[IMF.Country == 'Kosovo']

,Country,Classification,ISO Code
110,Kosovo,Emerging and Developing Economies,UVK


In [24]:
WB[WB.Economy == 'Kosovo']

,Economy,Code,Region,Income group
105,Kosovo,XKX,Europe & Central Asia,Upper middle income


* IMFとWorld BankではKosovoのコードが異なる

In [25]:
IMF['ISO Code'] = IMF['ISO Code'].str.replace('UVK','XKX')

In [26]:
countries = pd.merge(WB,IMF,left_on='Code',right_on='ISO Code',how='outer')

In [27]:
countries.head()

,Economy,Code,Region,Income group,Country,Classification,ISO Code
0,Afghanistan,AFG,South Asia,Low income,Afghanistan,Emerging and Developing Economies,AFG
1,Albania,ALB,Europe & Central Asia,Upper middle income,Albania,Emerging and Developing Economies,ALB
2,Algeria,DZA,Middle East & North Africa,Upper middle income,Algeria,Emerging and Developing Economies,DZA
3,American Samoa,ASM,East Asia & Pacific,Upper middle income,NaN,NaN,NaN
4,Andorra,AND,Europe & Central Asia,High income,NaN,NaN,NaN


In [28]:
countries['ISO Code'] = countries['ISO Code'].str.replace('XKX','UVK')

## United Nations

In [29]:
UN = pd.read_csv('UNSD_Methodology.csv')

In [30]:
UN.head()

,Global Code,Global Name,Region Code,Region Name,Sub-region Code,Sub-region Name,Intermediate Region Code,Intermediate Region Name,Country or Area,M49 Code,ISO-alpha3 Code,Least Developed Countries (LDC),Land Locked Developing Countries (LLDC),Small Island Developing States (SIDS),Developed / Developing Countries
0,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Algeria,12,DZA,NaN,NaN,NaN,Developing
1,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Egypt,818,EGY,NaN,NaN,NaN,Developing
2,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Libya,434,LBY,NaN,NaN,NaN,Developing
3,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Morocco,504,MAR,NaN,NaN,NaN,Developing
4,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Sudan,729,SDN,x,NaN,NaN,Developing


In [31]:
economies_in_transition = ['Albania',
                           'Bosnia and Herzegovina',
                           'Montenegro',
                           'North Macedonia',
                           'Serbia',
                           'Armenia',
                           'Azerbaijan',
                           'Belarus',
                           'Georgia',
                           'Kazakhstan',
                           'Kyrgyzstan',
                           'Republic of Moldova',
                           'Russian Federation',
                           'Tajikistan',
                           'Turkmenistan',
                           'Ukraine',
                           'Uzbekistan']

In [32]:
UN.loc[UN[UN['Country or Area'].isin(economies_in_transition)].index,'Developed / Developing Countries'] = 'Developing'

In [33]:
UN.head()

,Global Code,Global Name,Region Code,Region Name,Sub-region Code,Sub-region Name,Intermediate Region Code,Intermediate Region Name,Country or Area,M49 Code,ISO-alpha3 Code,Least Developed Countries (LDC),Land Locked Developing Countries (LLDC),Small Island Developing States (SIDS),Developed / Developing Countries
0,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Algeria,12,DZA,NaN,NaN,NaN,Developing
1,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Egypt,818,EGY,NaN,NaN,NaN,Developing
2,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Libya,434,LBY,NaN,NaN,NaN,Developing
3,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Morocco,504,MAR,NaN,NaN,NaN,Developing
4,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Sudan,729,SDN,x,NaN,NaN,Developing


In [34]:
UN.to_csv('UN_Classification.csv',index=False)